In [32]:
###############################################################################
#               Import part
###############################################################################
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import neighbors, model_selection
import seaborn as sns
from matplotlib.colors import ListedColormap
import pylab as pl
from sklearn.svm import SVC
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [2]:
###############################################################################
#               Data Generation
###############################################################################


def rand_gauss(n=100, mu=[1, 1], sigmas=[0.1, 0.1]):
    """ Sample n points from a Gaussian variable with center mu,
    and std deviation sigma
    """
    d = len(mu)
    res = np.random.randn(n, d)
    return np.array(res * sigmas + mu)


def rand_bi_gauss(n1=100, n2=100, mu1=[1, 1], mu2=[-1, -1], sigmas1=[0.1, 0.1],
                  sigmas2=[0.1, 0.1]):
    """ Sample n1 and n2 points from two Gaussian variables centered in mu1,
    mu2, with respective std deviations sigma1 and sigma2
    """
    ex1 = rand_gauss(n1, mu1, sigmas1)
    ex2 = rand_gauss(n2, mu2, sigmas2)
    y = np.hstack([np.ones(n1), -1 * np.ones(n2)])
    X = np.vstack([ex1, ex2])
    ind = np.random.permutation(n1 + n2)
    return X[ind, :], y[ind]

In [3]:
###############################################################################
#           Displaying labeled data
###############################################################################

symlist = ['o', 's', 'D', 'x', '+', '*', 'p', 'v', '-', '^']


def plot_2d(data, y=None, w=None, alpha_choice=1):
    """ Plot in 2D the dataset data, colors and symbols according to the
    class given by the vector y (if given); the separating hyperplan w can
    also be displayed if asked"""

    k = np.unique(y).shape[0]
    color_blind_list = sns.color_palette("colorblind", k)
    sns.set_palette(color_blind_list)
    if y is None:
        labs = [""]
        idxbyclass = [range(data.shape[0])]
    else:
        labs = np.unique(y)
        idxbyclass = [np.where(y == labs[i])[0] for i in range(len(labs))]

    for i in range(len(labs)):
        plt.scatter(data[idxbyclass[i], 0], data[idxbyclass[i], 1],
                    c=color_blind_list[i], s=80, marker=symlist[i])
    plt.ylim([np.min(data[:, 1]), np.max(data[:, 1])])
    plt.xlim([np.min(data[:, 0]), np.max(data[:, 0])])
    mx = np.min(data[:, 0])
    maxx = np.max(data[:, 0])
    if w is not None:
        plt.plot([mx, maxx], [mx * -w[1] / w[2] - w[0] / w[2],
                              maxx * -w[1] / w[2] - w[0] / w[2]],
                 "g", alpha=alpha_choice)

In [4]:
###############################################################################
#           Displaying tools for the Frontiere
###############################################################################


def frontiere(f, X, y, w=None, step=50, alpha_choice=1, colorbar=True,
              samples=True):
    """ trace la frontiere pour la fonction de decision f"""
    # construct cmap

    min_tot0 = np.min(X[:, 0])
    min_tot1 = np.min(X[:, 1])

    max_tot0 = np.max(X[:, 0])
    max_tot1 = np.max(X[:, 1])
    delta0 = (max_tot0 - min_tot0)
    delta1 = (max_tot1 - min_tot1)
    xx, yy = np.meshgrid(np.arange(min_tot0, max_tot0, delta0 / step),
                         np.arange(min_tot1, max_tot1, delta1 / step))
    z = np.array([f([vec]) for vec in np.c_[xx.ravel(), yy.ravel()]])
    z = z.reshape(xx.shape)
    labels = np.unique(z)
    color_blind_list = sns.color_palette("colorblind", labels.shape[0])
    sns.set_palette(color_blind_list)
    my_cmap = ListedColormap(color_blind_list)
    plt.imshow(z, origin='lower', interpolation="mitchell", alpha=0.80,
               cmap=my_cmap, extent=[min_tot0, max_tot0, min_tot1, max_tot1])
    if colorbar is True:
        ax = plt.gca()
        cbar = plt.colorbar(ticks=labels)
        cbar.ax.set_yticklabels(labels)

    labels = np.unique(y)
    k = np.unique(y).shape[0]
    color_blind_list = sns.color_palette("colorblind", k)
    sns.set_palette(color_blind_list)
    ax = plt.gca()
    if samples is True:
        for i, label in enumerate(y):
            label_num = np.where(labels == label)[0][0]
            plt.scatter(X[i, 0], X[i, 1], c=color_blind_list[label_num],
                        s=80, marker=symlist[label_num])
    plt.xlim([min_tot0, max_tot0])
    plt.ylim([min_tot1, max_tot1])
    ax.get_yaxis().set_ticks([])
    ax.get_xaxis().set_ticks([])
    if w is not None:
        plt.plot([min_tot0, max_tot0],
                 [min_tot0 * -w[1] / w[2] - w[0] / w[2],
                  max_tot0 * -w[1] / w[2] - w[0] / w[2]],
                 "k", alpha=alpha_choice)


In [5]:
def plot_gallery(images, titles, n_row=3, n_col=4):
    """Helper function to plot a gallery of portraits"""
    pl.figure(figsize=(1.8 * n_col, 2.4 * n_row))
    pl.subplots_adjust(bottom=0, left=.01, right=.99, top=.90,
                       hspace=.35)
    for i in range(n_row * n_col):
        pl.subplot(n_row, n_col, i + 1)
        pl.imshow(images[i])
        pl.title(titles[i], size=12)
        pl.xticks(())
        pl.yticks(())


def title(y_pred, y_test, names):
    pred_name = names[int(y_pred)].rsplit(' ', 1)[-1]
    true_name = names[int(y_test)].rsplit(' ', 1)[-1]
    return 'predicted: %s\ntrue:      %s' % (pred_name, true_name)

## SVM et noyaux pour la classification binaire

1) En vous basant sur la documentation, écrivez un code qui va classifier la classe 1 contre la classe 2 du dataset iris en utilisant les deux premières variables et un noyau linéaire. En laissant la moitié des données de côté, évaluez la performance en généralisation du modèle.

In [6]:
# load and scale data
scaler = StandardScaler() 
iris = datasets.load_iris()
X = iris.data 
X = scaler.fit_transform(X) 
y = iris.target 
X = X[y != 0, :2] 
y = y[y != 0]
X_train = X[0::2]
X_test = X[1::2]
y_train = y[0::2]
y_test = y[1::2]

In [7]:
# estimation sur l'échantillon train
svc = SVC(gamma='scale', kernel="linear")
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
# score
sc = svc.score(X_test, y_test)
print("Le score sur l'échantillon test est de " + str(sc) + ".")

Le score sur l'échantillon test est de 0.72.


2) Comparez le résultat avec un SVM basé sur noyau polynomial.

In [8]:
# estimation sur l'échantillon train
svc = SVC(gamma='scale', kernel="poly")
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
# score
sc = svc.score(X_test, y_test)
print("Le score sur l'échantillon test est de " + str(sc) + ".")

Le score sur l'échantillon test est de 0.6.


De façon surprenante, les résultats ne sont pas meilleurs avec un noyau polynomial. Il semblerait que les données ne soient pas bien représentées par ce type de noyau.

3) Montrez que le problème primal résolu par le SVM peut se réécrire sous une forme alternative:

On part du problème initial:

$(w^*, w_0^*, \xi^* \in R^n) \in \underbrace{argmin}_{w \in H, w_0 \in R, \xi \in R^n} \left( \frac{1}{2} ||w^2|| + C \sum_{i=1}^{n} \xi_i \right)$

s.c.  $\xi_i \geq 0, \ \forall i = 1 \ldots n, \hspace{10mm} y_i(<w, \Phi(x_i)> + w_0) \geq 1 - \xi_i, \ \forall i = 1 \ldots n$

La seconde contrainte implique:

$ \xi_i \geq 1 - y_i(<w, \Phi(x_i)> + w_0)$



On formule le Lagrangien:

$\mathcal{L}(w, w_0, \xi, \alpha, \mu) = \frac{1}{2} w^T w + C \sum_{i=1}^{n} \xi_i + \sum_{i=1}^{n} \alpha_i (1 - y_i (w^T x_i + w_0) - \xi_i) - \sum_{i=1}^{n} \mu_i \xi_i$

On prend les conditions de premier ordre:

$\frac{\partial \mathcal{L}}{\partial \xi_i} = 0 \Leftrightarrow C - \alpha_i - \mu_i = 0$

(trop long de tout réécrire en Latex. On ne fait ici que reprendre le contenu du cours. S'y référer.)

## Classification de visages

L’exemple suivant est un problème de classification de visages. La base de données à utiliser est disponible à l’adresse suivante : https://scikit-learn.org/stable/auto_examples/applications/ plot_face_recognition.html. Vous pouvez choisir deux personnes, par exemple Tony Blair et Colin Powell, pour accélérer le calcul. Aussi, vous pouvez utilisez seulement grayscale (et non le couleur) pour réduire le nombre de variables.

In [9]:
# load dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw_people.data
y = lfw_people.target

# retain only data for Powell (label = 1) and Blair (label = 6)
X_powell = X[y==1]
y_powell = y[y==1]
X_blair = X[y==6]
y_blair = y[y==6]
X = np.concatenate((X_powell, X_blair), axis=0)
y = np.concatenate((y_powell, y_blair), axis=0)
X_train = X[0::2]
X_test = X[1::2]
y_train = y[0::2]
y_test = y[1::2]

5) Utilisez les features centrées et réduites. (Pourquoi ?)

In [10]:
# scale
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
print("The mean of X_train is " + str(np.mean(X_train, axis=0)))
print("The standard deviation of X_train is " + str(np.std(X_train, axis=0)))
print(np.std(X_train))

The mean of X_train is [ 1.9763645e-08  1.1105286e-07 -1.2077783e-08 ...  8.1564249e-09
  1.1920929e-08  2.0704771e-08]
The standard deviation of X_train is [0.99999994 0.99999994 1.         ... 1.0000001  1.         1.        ]
0.99999994


6) Montrez l’influence du paramètre de régularisation. On pourra par exemple afficher l’erreur de prédiction en fonction de C sur une échelle logarithmique entre 1e5 et 1e-5.

In [11]:
# initiate grid search
parameters = {'C':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
svc = SVC(gamma='scale', kernel="rbf");
clf = GridSearchCV(svc, parameters);
clf.fit(X_train, y_train);

In [12]:
# collect all prediction errors and display
pred_errors = 1 - clf.cv_results_['mean_test_score']
for iter, value in enumerate([0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]):
    print("Prediction error with C = " + str(value) + " is " + str(round(pred_errors[iter], 3)))

Prediction error with C = 1e-05 is 0.379
Prediction error with C = 0.0001 is 0.379
Prediction error with C = 0.001 is 0.379
Prediction error with C = 0.01 is 0.379
Prediction error with C = 0.1 is 0.379
Prediction error with C = 1 is 0.168
Prediction error with C = 10 is 0.132
Prediction error with C = 100 is 0.132
Prediction error with C = 1000 is 0.132
Prediction error with C = 10000 is 0.132
Prediction error with C = 100000 is 0.132


De plus grandes valeurs de C donnent de meilleurs résultats. Il n'y a toutefois pas de gain significatif pour une valeur de C au-delà de 10.

In [18]:
# estimate model on X_train
svc = SVC(gamma='scale', kernel="linear")
svc.fit(X_train, y_train)
pred = svc.predict(X_test)
sc = svc.score(X_test, y_test)
print("Le score sur l'échantillon test est de " + str(round(sc, 3)) + ".")

Le score sur l'échantillon test est de 0.947.


7) En ajoutant des variables de nuisances (par exemple 300 variables normales centrées réduites), augmentant ainsi le nombre de variables à nombre de points d’apprentissage fixé, montrez que la performance chute.

In [28]:
# create noisy dataset
X_size, num_features = X.shape
noise = np.random.standard_normal(size=(X_size, 300))
X_noisy = np.concatenate((X, noise), axis=1)
X_train_noisy = X_noisy[0::2]
X_test_noisy = X_noisy[1::2]

array([[-1.96756858,  0.79861181,  0.94967103, ...,  0.73617345,
         1.48450338,  0.67603846],
       [ 0.23965237, -0.27440485,  0.39147991, ...,  0.9585222 ,
        -0.25879163, -0.3612656 ],
       [-0.89850811, -1.75992424,  3.3648917 , ...,  0.34381152,
         0.9269031 ,  0.78103585],
       ...,
       [ 0.88398359,  0.94289982,  2.54917003, ...,  0.67496613,
         0.76590123, -1.80760608],
       [-0.73962237, -0.94384347,  1.24710799, ...,  0.6814417 ,
         1.31853301,  1.58907436],
       [-1.16664067,  1.64751347,  1.47668756, ...,  0.41768046,
         2.05485397,  1.1598334 ]])

In [30]:
# estimate model on X_train_noisy
svc = SVC(gamma='scale', kernel="linear")
svc.fit(X_train_noisy, y_train)
pred = svc.predict(X_test_noisy)
sc = svc.score(X_test_noisy, y_test)
print("Le score sur l'échantillon test est de " + str(round(sc, 3)) + ".")

Le score sur l'échantillon test est de 0.953.


8) Vous pourrez améliorer la prédiction à l’aide d’une réduction de dimension basée sur l’objet sklearn.decomposition.PCA.

In [50]:
# PCA preserving 95% of data variance
pca = PCA(n_components = 0.95, svd_solver = 'full')
X_pca = pca.fit_transform(X_noisy)
X_train_pca = X_pca[0::2]
X_test_pca = X_pca[1::2]

In [51]:
# estimate model on X_train_noisy
svc = SVC(gamma='scale', kernel="linear")
svc.fit(X_train_pca, y_train)
pred = svc.predict(X_test_pca)
sc = svc.score(X_test_pca, y_test)
print("Le score sur l'échantillon test est de " + str(round(sc, 3)) + ".")

Le score sur l'échantillon test est de 0.937.
